In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
target = pd.read_csv("../data/target_train.csv")
sample = pd.read_csv("../data/sample_submission.csv")

plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

lom_train = pd.read_csv("../data/lom_train.csv")
lom_test = pd.read_csv("../data/lom_test.csv")

produv_train = pd.read_csv("../data/produv_train.csv")
produv_test = pd.read_csv("../data/produv_test.csv")

chronom_train = pd.read_csv("../data/chronom_train.csv")
chronom_test = pd.read_csv("../data/chronom_test.csv")

sip_train = pd.read_csv("../data/sip_train.csv")
sip_test = pd.read_csv("../data/sip_test.csv")

from pipeline import merge_data

params = {
    "chugun": {},
    "plavki": {"bow_count": 10},
    "vector_size": 10
}

train, test, y, num_features, cat_features = merge_data(
    sample, target, plavki_train, plavki_test, gas_train, gas_test, chugun_train, chugun_test, 
    lom_train, lom_test, produv_train, produv_test, chronom_train, chronom_test, 
    sip_train, sip_test, params
)

/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:76: RuntimeWarning: divide by zero encountered in true_divide
  lom_train_transformed['ves_loma/ves_chuguna'] = lom_train_transformed['ves_loma'].values/chugun_train['VES'].values
/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:82: RuntimeWarning: divide by zero encountered in true_divide
  lom_test_transformed['ves_loma/ves_chuguna'] = lom_test_transformed['ves_loma'].values/chugun_test['VES'].values


  0%|          | 0/2063 [00:00<?, ?it/s]

  0%|          | 0/780 [00:00<?, ?it/s]

In [3]:
from sklearn.model_selection import KFold, TimeSeriesSplit
from tqdm.auto import tqdm, trange

def metric(c_true, tst_true, c_pred, tst_pred, pwc=None, pwt=None):
    
    if pwc is not None:
        c_pred = pwc.inverse_transform(c_pred.reshape(-1, 1)).reshape(-1)
    if pwt is not None:
        tst_pred = pwt.inverse_transform(tst_pred.reshape(-1, 1)).reshape(-1)
    
    delta_c = np.abs(c_true - c_pred)
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(tst_true - tst_pred)
    hit_rate_t = np.int64(delta_t < 20)

    N = c_pred.shape[0]    
    return np.sum(hit_rate_c) / N, np.sum(hit_rate_t) / N, np.sum(hit_rate_c + hit_rate_t) / 2 / N

def pipeline(model_c, model_tst, train_c, test_c, train_tst, test_tst, y, sample, n_splits=10, pwc=None, pwt=None):
    kf = KFold(n_splits=n_splits, shuffle=True)

    sample["C"] = 0
    sample["TST"] = 0
    
    res_c, res_t, res = [], [], []
    for train_idx, test_idx in tqdm(kf.split(train_c), total=n_splits):
        cur_train_c = train_c[train_idx]
        cur_eval_c = train_c[test_idx]        

        cur_train_tst = train_tst[train_idx]
        cur_eval_tst = train_tst[test_idx]

        cur_train_y = y.iloc[train_idx]
        cur_eval_y = y.iloc[test_idx]
        
        if pwc is not None:
            model_c.fit(cur_train_c, pwc.transform(cur_train_y["C"].values.reshape(-1, 1)).reshape(-1))
        else:
            model_c.fit(cur_train_c, cur_train_y["C"])
            
        if pwt is not None:
            model_tst.fit(cur_train_tst, pwt.transform(cur_train_y["TST"].values.reshape(-1, 1)).reshape(-1))
        else:
            model_tst.fit(cur_train_tst, cur_train_y["TST"])
        
        eval_pred_c = model_c.predict(cur_eval_c)
        eval_pred_tst = model_tst.predict(cur_eval_tst)
    
        sample["C"] += model_c.predict(test_c) / n_splits
        sample["TST"] += model_tst.predict(test_tst) / n_splits
        
        hit_rate_c, hit_rate_t, hit_rate = metric(cur_eval_y["C"], cur_eval_y["TST"], eval_pred_c, eval_pred_tst, pwc, pwt)
        res_c.append(hit_rate_c)
        res_t.append(hit_rate_t)
        res.append(hit_rate)
    
    res_c = np.array(res_c)
    res_t = np.array(res_t)
    res = np.array(res)
    print(f"Carbon score: {res_c.mean()} ± {res_c.std()}")
    print(f"Temperature score: {res_t.mean()} ± {res_t.std()}")    
    print(f"Overall score: {res.mean()} ± {res.std()}") 
    
    if pwt is not None:
        sample["TST"] = pwt.inverse_transform(sample["TST"].values.reshape(-1, 1)).reshape(-1)
    if pwc is not None:
        sample["C"] = pwc.inverse_transform(sample["C"].values.reshape(-1, 1)).reshape(-1)
    
    sample["C"] = sample["C"].clip(0, 1)
    
    return sample

In [17]:
from sklearn.preprocessing import PowerTransformer

pwrC = PowerTransformer('box-cox')
pwrT = PowerTransformer('box-cox')

pwrC2 = PowerTransformer()
pwrT2 = PowerTransformer()

y_exp = y.copy()

y_exp["C"] = pwrC.fit_transform(y['C'].values.reshape(-1, 1))
y_exp["TST"] = pwrT.fit_transform(y['TST'].values.reshape(-1, 1))

pwrC2.fit(y['C'].values.reshape(-1, 1))
pwrT2.fit(y['TST'].values.reshape(-1, 1))

PowerTransformer()

In [42]:
train_backup = train.copy()
test_backup = test.copy()

In [48]:
train = train_backup.copy()
test = test_backup.copy()

In [8]:
gas_filtered_for_C = pd.read_csv("../gas/gas_filtered_for_C.csv").set_index("NPLV")
gas_filtered_for_TST = pd.read_csv("../gas/gas_filtered_for_TST.csv").set_index("NPLV")

gas_test_autofeatures = pd.read_csv("../gas/gas_test_autofeatures.csv").set_index("NPLV")

In [9]:
train_gas_C = gas_filtered_for_C
train_gas_TST = gas_filtered_for_TST

test_gas_C = gas_test_autofeatures[gas_filtered_for_C.columns]
test_gas_TST = gas_test_autofeatures[gas_filtered_for_TST.columns]

In [49]:
train_C = train.join(train_gas_C, on="NPLV")
train_TST = train.join(train_gas_TST, on="NPLV")

test_C = test.join(test_gas_C, on="NPLV")
test_TST = test.join(test_gas_TST, on="NPLV")

In [46]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

model_c = XGBRegressor(n_estimators=40, n_jobs=-1)
model_tst = XGBRegressor(n_estimators=40, n_jobs=-1)

# model_c = RandomForestRegressor(n_estimators=50, n_jobs=-1)
# model_tst = RandomForestRegressor(n_estimators=50, n_jobs=-1)

res = pipeline(model_c, model_tst, train_C.values, test_C.values, train_TST.values, test_TST.values, y, sample, 
               n_splits=50, pwc=pwrC, pwt=pwrT)

  0%|          | 0/50 [00:00<?, ?it/s]

Carbon score: 0.7371660859465738 ± 0.06580878338775255
Temperature score: 0.6718118466898956 ± 0.07481081467917064
Overall score: 0.7044889663182348 ± 0.05393976342579064


In [50]:
model_c = XGBRegressor(n_estimators=50, n_jobs=-1)
model_c.fit(train_C, y["C"])

model_tst = XGBRegressor(n_estimators=50, n_jobs=-1)
model_tst.fit(train_TST, y["TST"])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [52]:
train_C.to_csv("train_C.csv", index=False)
train_TST.to_csv("train_TST.csv", index=False)

y.to_csv("y.csv", index=False)

,gas_duration,gas_mean_V,gas_mean_T,gas_mean_O2,gas_mean_N2,gas_mean_H2,gas_mean_CO2,gas_mean_CO,gas_mean_AR,gas_mean_T фурмы 1,gas_mean_T фурмы 2,gas_mean_O2_pressure,gas_std_V,gas_std_T,gas_std_O2,gas_std_N2,gas_std_H2,gas_std_CO2,gas_std_CO,gas_std_AR,gas_std_T фурмы 1,gas_std_T фурмы 2,gas_std_O2_pressure,gas_sum_volume_O2,gas_sum_volume_N2,gas_sum_volume_H2,gas_sum_volume_CO2,gas_sum_volume_CO,gas_sum_volume_AR,gas_mean_volume_O2,gas_mean_volume_N2,gas_mean_volume_H2,gas_mean_volume_CO2,gas_mean_volume_CO,gas_mean_volume_AR,gas_std_volume_O2,gas_std_volume_N2,gas_std_volume_H2,gas_std_volume_CO2,gas_std_volume_CO,gas_std_volume_AR,dayofmonth,hour,duration,truncated_NMZ,st_diff_is_zero,dayofweek,plavka_TIPE_GOL,plavka_TIPE_FUR,plavka_NAPR_ZAD,bow_.z0,bow_/Э,bow_/ЭТ,bow_3пс,bow_SC2,bow_Ст3,bow_ЭТ,bow_пс/,bow_с/Э,bow_т3п,VES,T,SI,MN,S,P,CR,NI,CU,V,TI,si_portion,mn_portion,s_portion,p_portion,cr_portion,ni_portion,cu_portion,v_portion,ti_portion,ves_loma,ves_loma/ves_chuguna,durationproduv_,RAS_mean,POL_mean,0,1,2,3,4,5,6,7,8,9,O2_вн.пл.прост._0,O2_межпл.прост._0,O2_межпл.прост._1,O2_опер_0,total_duration_вн.пл.прост._0,total_duration_межпл.прост._0,total_duration_межпл.прост._1,total_duration_опер_0,min_duration_вн.пл.прост._0,min_duration_межпл.прост._0,min_duration_межпл.прост._1,min_duration_опер_0,max_duration_вн.пл.прост._0,max_duration_межпл.прост._0,max_duration_межпл.прост._1,max_duration_опер_0,total_operations_вн.пл.прост._0,total_operations_межпл.прост._0,total_operations_межпл.прост._1,total_operations_опер_0,min_mass,max_mass,total_count,unique_count,min_ratio,max_ratio,unique_ratio,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,w2v_10,w2v_11,w2v_12,w2v_13,w2v_14,w2v_15,w2v_16,w2v_17,w2v_18,w2v_19,w2v_20,w2v_sip_0,w2v_sip_1,w2v_sip_2,w2v_sip_3,w2v_sip_4,w2v_sip_5,w2v_sip_6,w2v_sip_7,w2v_sip_8,w2v_sip_9,w2v_sip_10,w2v_sip_11,w2v_sip_12,w2v_sip_13,w2v_sip_14,w2v_sip_15,w2v_sip_16,w2v_sip_17,w2v_sip_18,w2v_sip_19,w2v_sip_20,"CO__fft_coefficient__attr_""imag""__coeff_1","T__fft_coefficient__attr_""imag""__coeff_1","T фурмы 2__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","CO2__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","T__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","CO2__fft_coefficient__attr_""abs""__coeff_25",AR__energy_ratio_by_chunks__num_segments_10__segment_focus_7,"T фурмы 2__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4",N2__quantile__q_0.9,CO2__sum_values,CO__energy_ratio_by_chunks__num_segments_10__segment_focus_6,CO2__energy_ratio_by_chunks__num_segments_10__segment_focus_9,T__index_mass_quantile__q_0.6,"CO2__fft_coefficient__attr_""abs""__coeff_57","O2__fft_coefficient__attr_""abs""__coeff_68","CO__fft_coefficient__attr_""imag""__coeff_2",CO2__index_mass_quantile__q_0.8,"T фурмы 2__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6","N2__fft_coefficient__attr_""abs""__coeff_65","CO2__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","T фурмы 2__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6","CO2__fft_coefficient__attr_""abs""__coeff_16",T__absolute_sum_of_changes,"T__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","O2__fft_coefficient__attr_""abs""__coeff_52",N2__quantile__q_0.7,"T__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""",CO2__index_mass_quantile__q_0.9,"V__fft_coefficient__attr_""abs""__coeff_13",CO__index_mass_quantile__q_0.6,T__last_location_of_maximum,AR__sum_of_reoccurring_values,CO__energy_ratio_by_chunks__num_segments_10__segment_focus_8,AR__quantile__q_0.9,"CO2__fft_coefficient__attr_""abs""__coeff_27","CO2__fft_coefficient__attr_""real""__coeff_2","CO2__fft_coefficient__attr_""imag""__coeff_2",AR__autocorrelation__lag_2,"N2__fft_aggregated__aggtype_""variance""","T фурмы 2__fft_coefficient__attr_""abs""__coeff_13",AR__autocorrelation__lag_7,"T фурмы 2__fft_coefficient__attr_""imag""__coeff_2","O2_pressure__fft_coefficient__attr_""real""__c

In [51]:
sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:50]

[(0.34396577, 'truncated_NMZ'),
 (0.10533005, 'CO__index_mass_quantile__q_0.6'),
 (0.07750683, 'total_operations_опер_0'),
 (0.0370315, 'T__absolute_sum_of_changes'),
 (0.02811455, 'T__index_mass_quantile__q_0.6'),
 (0.025079371, 'unique_count'),
 (0.023421153, 'min_ratio'),
 (0.018794352, 'N2__fft_aggregated__aggtype_"variance"'),
 (0.016543178, 'gas_mean_T фурмы 2'),
 (0.012752775, 'w2v_sip_4'),
 (0.011344053, 'CO__fft_coefficient__attr_"imag"__coeff_2'),
 (0.011310781,
  'T фурмы 2__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6'),
 (0.010854029, 'N2__quantile__q_0.7'),
 (0.010292539, 'w2v_18'),
 (0.009005749, 'AR__autocorrelation__lag_8'),
 (0.0084295655, 'w2v_sip_19'),
 (0.007422591,
  'T фурмы 2__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4'),
 (0.007261362, 'T__fft_coefficient__attr_"imag"__coeff_1'),
 (0.007218814, 'VES'),
 (0.006763379, 'N2__quantile__q_0.9'),
 (0.006727357, 'max_mass'),
 (0.0063897823, 'CO2__sum_values'),
 (0.0063811927, 'gas_sum_volu

In [47]:
res.to_csv("xgb_no_NMZ.csv", index=False)